# Necessary Imports

In [2]:
!pip install transformers torch onnx openvino-dev[onnx] datasets accelerate openvino peft bitsandbytes trl -U

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached onnx-1.16.2-cp310-cp310-win_amd64.whl.metadata (16 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


In [3]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: d:\anaconda\envs\gpu\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl
Name: accelerate
Version: 0.33.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: d:\anaconda\envs\gpu\lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


In [4]:
!pip uninstall -y transformers accelerate
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)


In [1]:
import pandas as pd
import torch
from datasets import Dataset
from random import randrange
#from peft import Loraconfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

d:\Anaconda\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Save Pre-trained Model and Tokenizer

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_7b", use_fast=False, legacy=False)
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_7b")

model.save_pretrained("./my_open_llama_model")
tokenizer.save_pretrained("./my_open_llama_model")

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /openlm-research/open_llama_7b/resolve/main/model.safetensors.index.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D4571B4F10>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: a9965020-07d3-42d3-a569-dc822b178dd2)')

# Convert PyTorch Model to ONNX Format

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./my_open_llama_model")
model = AutoModelForCausalLM.from_pretrained("./my_open_llama_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "open_llama_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=14,
                  do_constant_folding=True)

print("Model converted to ONNX format successfully.")

# Convert ONNX Model to OpenVINO IR Format

In [ ]:
import openvino as ov
import os

ir_model = ov.convert_model("open_llama_model.onnx",
                            input=[ov.PartialShape([1, 4])])

ov.save_model(ir_model, "open_llama_ir_model.xml")

print("Model converted to IR format successfully.")

print("XML file exists:", os.path.exists("open_llama_ir_model.xml"))
print("BIN file exists:", os.path.exists("open_llama_ir_model.bin"))

# Pre Processing Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('medquad.csv')

df = df[['question', 'answer']]

In [ ]:
train_val_df, sample_df = train_test_split(df, test_size=0.1, random_state=42) #using only sample df as the whole dataset is very big

In [ ]:
sample_df['text'] = 'Question:\n' + sample_df['question'] + '\n\nAnswer:\n' + sample_df['answer']
sample_df.drop(columns=['question','answer'], axis=1,inplace=True)
# sample_df.to_csv('/content/pre_dataset.csv', index=False)

In [ ]:
df = pd.read_csv('pre_dataset.csv')

train = Dataset.from_pandas(df)
train_dataset_size = len(train)
print(train_dataset_size)

In [ ]:
train

# Load and Prepare Model, Tokenizer for finetuning

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, DatasetDict
import torch


model_name = "./open_llama_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

train_dataset_size = len(train)
batch_size = 4
steps_per_epoch = train_dataset_size // batch_size

target_steps = 300
num_epochs = max(3, target_steps // steps_per_epoch)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
)

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    dataset_text_field='text',
    max_seq_length=1042,
    tokenizer=tokenizer,
    packing=True,
    callbacks=[PrinterCallback()]
)

trainer.train()

trainer.save_model("./fine_tuned_open_llama_model")

# Exporting the Fine-Tuned GPT-Neo Model to ONNX Format

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_open_llama_model")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_open_llama_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "fine_tuned_open_llama_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Fine-tuned model converted to ONNX format successfully.")

# Converting the Fine-Tuned GPT-Neo Model to OpenVINO IR Format

In [ ]:
from openvino.tools import mo

ov_model = mo.convert_model("fine_tuned_open_llama_model.onnx",
                            input=[('input_ids', [1, -1]), ('attention_mask', [1, -1])],
                            output=['output'])

from openvino.runtime import serialize
serialize(ov_model, "fine_tuned_distil_ir_model.xml")

print("Fine-tuned model converted to IR format successfully.")

# OpenVINO IR Format Fine Tuned Model Inference

In [ ]:
import openvino.runtime as ov
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_open_llama_model")

core = ov.Core()
model = core.read_model("fine_tuned_distil_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")

infer_request = compiled_model.create_infer_request()

def generate_text(prompt, max_length=200): # The maximum length is 200; the answer may be short.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]

        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

prompt = "What is (are) Schwannomatosis ?"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)

#True answer: Schwannomatosis is a rare form of neurofibromatosis that is primarily characterized by multiple schwannomas (benign tumors of the nervous system) in the absence of bilateral (affecting both sides) vestibular schwannomas. Signs and symptoms of the condition vary based on the size, location and number of schwannomas but may include pain; numbness; tingling; and/or weakness in the fingers and toes. Inherited forms of the disorder account for only 15 percent of all cases. In some of these families, schwannomatosis is caused by changes (mutations) in the SMARCB1 or LZTR1 genes; in other cases, the exact underlying cause is unknown. When inherited, the condition is passed down in an autosomal dominant manner with highly variable expressivity and reduced penetrance. Treatment is based on the signs and symptoms present in each person but may include medications and/or surgery.

# Simple Fine Tuned Model Inference

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

fine_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_open_llama_model")
fine_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_open_llama_model")

In [ ]:
fine_model.cuda() # Using GPU for generating text

prompt = "What are the symptoms of Crome syndrome ?"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inputs = fine_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs.input_ids.to(device)
attention_mask = inputs.attention_mask.to(device)

outputs = fine_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=200,
    temperature=0.6,
    pad_token_id=fine_tokenizer.eos_token_id
)

In [ ]:
generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
import time
from sklearn.metrics import accuracy_score
import psutil
import torch
from nltk.translate.bleu_score import sentence_bleu

# 1. Inference Time (Latency)
start_time = time.time()
generated_text = generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")

In [ ]:
# 2. Throughput
num_samples = 10
start_time = time.time()
for _ in range(num_samples):
    generated_text = generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
end_time = time.time()
throughput = num_samples / (end_time - start_time)
print(f"Throughput: {throughput:.4f} samples/second")

In [ ]:
# 3. Model Load Time
start_time = time.time()
core = ov.Core()
model = core.read_model("fine_tuned_distil_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")
end_time = time.time()
model_load_time = end_time - start_time
print(f"Model Load Time: {model_load_time:.4f} seconds")

In [ ]:
# 4. CPU Utilization
cpu_percent = psutil.cpu_percent(interval=1)  # Check CPU utilization over 1 second
print(f"CPU Utilization: {cpu_percent}%")

In [ ]:
# 5. BLEU Score
question = "What are the symptoms of Crome syndrome ?"
answer = """What are the signs and symptoms of Crome syndrome? The Human Phenotype Ontology provides the following list of signs and symptoms for Crome syndrome.
If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Abnormality of the renal tubule 90% Aplasia/Hypoplasia of the cerebellum 90% Cataract 90% Cognitive impairment 90% Encephalitis 90% Seizures 90% Nystagmus 50% Acute tubular necrosis -
Autosomal recessive inheritance - Cerebellar dysplasia - Congenital cataract - Encephalopathy - Intellectual disability - Short stature - The Human Phenotype Ontology (HPO) has collected information on how often a sign or symptom occurs in a condition. Much of this information comes from Orphanet, a European rare disease database.
 The frequency of a sign or symptom is usually listed as a rough estimate of the percentage of patients who have that feature. The frequency may also be listed as a fraction.
 The first number of the fraction is how many people had the symptom, and the second number is the total number of people who were examined in one study. For example, a frequency of 25/25 means that in a study of 25 people all patients were found to have that symptom.
 Because these frequencies are based on a specific study, the fractions may be different if another group of patients are examined. Sometimes, no information on frequency is available. In these cases, the sign or symptom may be rare or common."""

def generate_text_openvino(prompt, max_length=200):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]
        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

generated_text = generate_text_openvino(question)
reference = [answer.split()]
candidate = generated_text.split()
bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score}")


In [ ]:
# 6. Fine-tuning Time
start_time = time.time()
trainer.train()
end_time = time.time()
fine_tuning_time = end_time - start_time
print(f"Fine-tuning Time: {fine_tuning_time:.4f} seconds")

In [ ]:
metrics = {
    "Inference Time (Latency)": inference_time,
    "Throughput": throughput,
    "Model Load Time": model_load_time,
    "CPU Utilization": cpu_percent,
    "BLEU Score": bleu_score,
    "Fine-tuning Time": fine_tuning_time
}

with open("metrics.txt", "w") as f:
    for key, value in metrics.items():
        f.write(f"{key}: {value}\n")

print("Metrics saved to metrics.txt")